# ShopAssist AI


## Part 1: Introduction



`
#### Project Background

In today's digital age, online shopping has become the go-to option for many consumers. However, the overwhelming number of choices and the lack of personalized assistance can make the shopping experience daunting. To address this, we have developed **ShopAssist AI, a chatbot that combines the power of large language models and rule-based functions to ensure accurate and reliable information delivery**.


#### Problem Statement

*Given a dataset containing information about laptops (product names, specifications, descriptions, etc.), build a chatbot that parses the dataset and provides accurate laptop recommendations based on user requirements*.


You can load the data and see it here.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.1/387.1 kB 7.5 MB/s eta 0:00:00


Mounted at /content/drive


In [3]:
# Import other libraries
import pandas as pd
from IPython.display import display, HTML

pd.set_option("display.width", 100)

In [4]:
filepath = "/content/drive/MyDrive/GenAI/"

#### Approach:

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play, extracting top 3 laptops that best matches the user's needs.
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect laptop solution.

## Part 2: System Design


#### Dataset

We have a dataset `laptop.csv` where  each row describes the features of a single laptop and also has a small description at the end. The chatbot that we build will leverage LLMs to parse this `Description` column and provide recommendations

#### Workings of the Chatbot



The chatbot should ask a series of questions to
- Determine the user's requirments. For simplicity, we have used 6 features to encapsulate the user's needs. The 6 features are as follows:
    - GPU intensity
    - Display quality
    - Portability
    - Multitasking
    - Processing speed
    - Budget

- Confirm if the user's requirements have been correctly captured at the end.

After that the chatbot lists down the top 3 products that are the most relevant, and engages in further conversation to help the user find the best one.


# ShopAssist 1.0

#### Building the Chatbot


|`Stage 1`

- Intent Clarity Layer
- Intent Confirmation Layer

`Stage 2`

- Product Mapping Layer
- Product Information Extraction Layer

`Stage 3`

- Product Recommendation Layer

##### Major functions behind the Chatbot

Let's now look at a brief overview of the major functions that form the chatbot. We'll take a deep dive later



- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget
- `dictionary_present()`: This function checks if the final understanding of user's profile is returned by the chatbot as a python dictionary or not. If there is a dictionary, it extracts the information as a Python dictionary.
- `compare_laptops_with_user()`: This function compares the user's profile with the different laptops and come back with the top 3 recommendations.
- `initialize_conv_reco()`: Initializes the recommendations conversation

In the next sections, we will look at how to write the code for the above functions.

# ShopAssist 2.0

But in development of **ShopAssist2.0** , we leverage from Function Calling feature for improved performance and simple architecture.

The architecture looks as shown below:

|`Stage 1`

- Intent Clarity Layer

`Stage 2`

- Product Mapping Layer

`Stage 3`

- Product Recommendation Layer

The following layers from ShopAssist 1.0 **are removed** as their purpose are filled by now using function calling in OpenAI API.

1. intent_confirmation_layer()
2. dictionary_present()
3. initialize_conv_reco()

## Stage 1

### 3.1 - Import the libraries

Let's start by importing the libraries that we'll require for this project. Following are the ones:
- openai
- pandas
- os, json, ast

Make sure the api key is stored in the text file `OPENAI_API_Key.txt`.

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Import the libraries
import json, ast
from tenacity import retry, wait_random_exponential, stop_after_attempt
import pandas as pd

load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

In [2]:
df = pd.read_csv("laptop_data.csv")

In [3]:
df.head(10)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...
5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.2 kg,Dual Cooling Fans,1 year,5 hours,"80,000",The Acer Predator is a powerhouse laptop desig...
6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8GB,SSD,PixelSense,"13.5""",Intel Iris Plus,2256x1504,Windows 11,1.3 kg,Touchscreen Display,1 year,10 hours,"90,000",The Microsoft Surface Laptop is a premium devi...
7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.6 kg,Backlit Keyboard,3 years,6 hours,"60,000",The Lenovo ThinkPad is a powerful laptop desig...
8,HP,Pavilion,i5,Intel,2.3 GHz,12GB,HDD,LCD,"15.6""",Intel UHD,1366x768,Windows 10,2.1 kg,B&O Audio,1 year,4 hours,"30,000",The HP Pavilion is a budget-friendly laptop th...
9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16GB,SSD,IPS,"17.3""",NVIDIA RTX,1920x1080,Windows 10,2.9 kg,Aura Sync RGB Keyboard,2 years,5 hours,"85,000",The ASUS ROG Strix G is a high-performance gam...


### Initialize Conversation

In [4]:
client = OpenAI(api_key=api_key)

In [16]:
# initialize_conversation function
def initialize_conversation_0():
    """
    Returns a list [{'role':'system','content': system_message}]
    """
    delimiter = "####"

    system_message = f"""

  You are very intelligent laptop gadget expert and your goal is to find out best laptop for a user for given input.
  You need to ask relevant questions to the user and understand the user profile  by analysing the user's responses.
  Your final objective is to get following parameters from user's conversation. The parameters are 'GPU intensity','Display quality','Portability','Multitasking','Processing speed' and 'Budget', and be confident of the values.
  You need to ask probing questions to the user in case any of these parameters are not obtained from user's interaction.
  The value for the 'Budget' should be numerical value extracted from user's response.
  The value of all keys except 'Budget' should be either 'high' or 'medium' or 'low' based on the importance of the corresponding keys as stated by user.
  All the values in the example dictionary are only representative values.
  {delimiter}
  Here are some instructions around the values for the different keys. if you do not follow this, you will be heavily penalized.
  - The value of all keys except 'Budget' should be strictly either 'high' or 'medium' or 'low' based on the importance of the corresponding keys as stated by user.
  - The value for the 'Budget' should be numerical value extracted from user's response.
  - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention to user that there are no laptops in that range.
  - Please do not randomly assign any value to the parameters.
  - The value needs to be inferred from the user's response.
  - Please ask one question at a time to capture values for the parameters.
  {delimiter}

  Once you have obtained all the parameter, your goal is to extract details of the top 3 laptops matching to the parameters obtaned earlier in the user's conversation .
  And this you will do by function calling with compare_laptops_with_user function.

  {delimiter}
    Once you get the list of top 3 laptops you will need to present the details in the personlized format it and show the recomendations to the
    user in the following format:
    1. <Laptop name>: <Basic laptop specs in brief>, <price of laptop>
    2. <Laptop name>: <Basic laptop specs in brief>, <price of laptop>
    3. <Laptop name>: <Basic laptop specs in brief>, <price of laptop>
  {delimiter}

  Follow below steps when interacting with user.


  Step 1 : Start with a short welcome message and encourage the user to share their requirement precisely on the laptop needed. Remember, you only recommend on laptops, if user is asking anything else then you apologizes and remind user that you are laptop expert only. Ask only one question at a time


  Step 2 : Based on the user input obtained from conversation, you try to get values for 'GPU intensity','Display quality','Portability','Multitasking','Processing speed' and 'Budget'.


  Step 3 : In case, details are not clear , ask clarifying question to user to get the details for above parameters.

  Step 4 : When all the parameters are available you invoke function calling with compare_laptops_with_user function.

  Step 5 : Personalize the recommendation obtained from compare_laptops_with_user function and present to the user a single laptop.

  Step 6 : If user changes any of the request then repeat step 2 to 5.


  {delimiter}
  Here are some notes which you should take care of. If you do not folow this you will be heavily penalized.
  - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
  - The value for 'budget' should be a numerical value extracted from the user's response. It needs to be greater than or equal to 25000 INR.
  - Do not ask questions to user for more 1 parameter at a time.
  - Do not ask too many questions in one shot.
  - Do not reply on any other subject other than laptop and its parameters.


    {delimiter}
  Her is a sample conversation between user and AI assistant:

  User : 'Hi, I am an editor.'
  Assistant : 'Great! As an editor, you likely require a laptop tha can handle demanding tasks. hence , the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on ? Are you more involved in video editing , photo editing or both ? Understanding the specific type of editing work will help me to tailor my recommendation accordingly. Let me know if my understanding is correct until now.  '

  User : ' I primarily work with after effects. '
  Assistant : ' Thank you for providing that information. Working with after effects involves working with graphics, animations and rendering which will require high GPU. Do you work with high resolution media files, such as 4K videos or RAW photos ? Understanding your file sizes will help determine the stoprage capacity and processing power needed. '

  User : ' Yes, sometimes I work with 4K videos as well. '
  Assistant : ' Thank you for the information. Processing 4K videos will require a good and high GPU. I think we have already determined earlier that you need a high CPU. To ensure that I have a complete understanding of your needs, I have one more question : Are you frequently on the go and require a laptop that s light weight and easy to carry, or do you primarily work from a stationary location ?  '

  User : ' Yes, sometimes I travel but do not carry my laptop. '
  Assistant : ' Could you kindly let me know your budget for the laptop ? This will help me to find the options that fit within your price range while meeting the specified requirements. '

  User : ' My maximum budget is 1.5 lakh INR. '

  {delimiter}

  {delimiter}
  Here is another sample conversation between the user and assistant:

  User: "I am a coder and i need higher end laptop with high multitasking feature"
  Assistant: "Great! As a coder, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. Could you please specify the importance of GPU intensity, display quality, portability, processing speed, and budget in order to tailor my recommendation accordingly?"

  User: "Yes, you are right"
  Assistant: "Great! Do you prefer a laptop that is easy to carry around or do you primarily use it at a fixed location?"

  User: "I only need it for home use."
  Assistant: "Thank you for specifying that. Do you often have multiple tasks running on your laptop simultaneously or primarily focus on one task at a time?"

  User: "I usually have multiple tasks running"
  Assistant: "Thank you for sharing that. Do you prefer a laptop with fast processing speed for quick performance?"

  User: "yes i need it to be very quick"
  Assistant: "Appreciate that input. Finally, what is your budget range for the laptop you are looking to purchase?"

  User: "i would like if it is not more than 125000 INR"


  {delimiter}
  Here is another sample conversation between the user and assistant:
  User: "I am a basic business user"
  Assistant: "As a basic business user, it's important to have a reliable laptop for your day-to-day tasks. Based on your usage as a basic business user, do you prefer a laptop with a good quality display for presentations?"
  User: "Yes, you right"
  Assistant: "Great! Do you prefer a laptop that is easy to carry around or do you primarily use it at a fixed location?"
  User: "I need it to be easy to carry around as I travel frequently on business trips"
  Assistant: "Thank you for specifying that. Do you often have multiple tasks running on your laptop simultaneously or primarily focus on one task at a time?"
  User: "i ususally have multiple tasks running"
  Assistant: "Thank you for sharing that. Do you prefer a laptop with fast processing speed for quick performance?"
  User: "yes i need it to be very quick"
  Assistant: "Appreciate that input. Finally, what is your budget range for the laptop you are looking to purchase?"
  User: "i would like if it is not more than 1 lakh"


  """

    conversation = [{"role": "system", "content": system_message}]

    return conversation

In [75]:
def initialize_conversation():
    """
    Returns a list [{"role": "system", "content": system_message}]
    """

    delimiter = "####"

    example_user_dict = {
        "GPU intensity": "high",
        "Display quality": "high",
        "Portability": "low",
        "Multitasking": "high",
        "Processing speed": "high",
        "Budget": "150000",
    }

    example_user_req = {
        "GPU intensity": "_",
        "Display quality": "_",
        "Portability": "_",
        "Multitasking": "_",
        "Processing speed": "_",
        "Budget": "_",
    }

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    All the values in the example dictionary are only representative values.
    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised:
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'Budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask followup questions to understand their needs.
    You are trying to fill the values of all the keys {{'GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget'}} in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their profile clearly. \n

    {delimiter}
    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_dict}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [18]:
df.to_json("laptop.jsonl")

### `get_chat_completions()`:

This function perform LLM call using the Chat Completions API to get the LLM response.

In [76]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format=False):
    MODEL = "gpt-3.5-turbo"

    system_message_json_output = (
        """<<. Return output in JSON format to the key output.>>"""
    )

    # If the output is required to be in JSON format
    if json_format:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]["content"] += system_message_json_output

        # JSON return type specified
        chat_completion_json = client.chat.completions.create(
            model=MODEL,
            messages=input,
            response_format={"type": "json_object"},
            seed=42,
        )

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = client.chat.completions.create(
            model=MODEL, messages=input, seed=42
        )

        output = chat_completion.choices[0].message.content

    return output

In [77]:
input_prompt = "I need a laptop for gaming"
messages = [{"role": "user", "content": input_prompt}]
# system_message_json_output = """<<. Return output in JSON format.>>"""
# messages[0]['content']+=system_message_json_output
messages

[{'role': 'user', 'content': 'I need a laptop for gaming'}]

In [78]:
debug_response_assistant = get_chat_completions(
    initialize_conversation(), json_format=True
)

In [80]:
display(debug_response_assistant)

{'GPU intensity': 'high',
 'Display quality': 'high',
 'Portability': 'low',
 'Multitasking': 'high',
 'Processing speed': 'high',
 'Budget': '150000'}

In [81]:
debug_response_assistant1 = get_chat_completions(initialize_conversation())

In [85]:
from IPython.display import display, Markdown

display(Markdown(debug_response_assistant1))

Hello! Welcome to our laptop recommendation service. I am here to assist you in finding the best laptop based on your requirements. Please share with me your needs and preferences so that I can help you make an informed decision.

In [96]:
def iterate_llm_response(funct, debug_response, num=2):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print("-" * 50)  # Print a separator line for readability
        i += 1  # Increment the counter

In [87]:
# Define a function called moderation_check that takes user_input as a parameter.


def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = client.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if moderation_output:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

### intent_confirmation_layer

This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not
   - GPU intensity
   - Display quality
   - Portability
   - Multitasking
   - Processing speed
   - Budget

In [97]:
def intent_confirmation_layer(response_assistant):

    delimiter = "####"

    allowed_values = {"low", "medium", "high"}

    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    {{
    'GPU intensity': 'values',
    'Display quality':'values',
    'Portability':'values',
    'Multitasking':'values',
    'Processing speed':'values',
    'Budget':'number'
    }}
    The values for the keys should only be from the allowed values: {allowed_values}.
    The 'Budget' key can take only a numerical value.
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    THought 3 - Think carefully before the answering.
    """

    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"""Here is the input: {response_assistant}"""},
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        response_format={"type": "json_object"},
        seed=42,
    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

### `dictionary_present()`:

This function checks if the final understanding of user's profile is returned by the chatbot is a Python dictionary or not. This is important as it'll be used later on for finding the right laptops using dictionary matching.

In [98]:
def dictionary_present(response):
    delimiter = "####"

    user_req = {
        "GPU intensity": "high",
        "Display quality": "high",
        "Portability": "medium",
        "Multitasking": "high",
        "Processing speed": "high",
        "Budget": "200000",
    }

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
            'GPU intensity': 'low/medium/high ',
            'Display quality':'low/medium/high',
            'Portability':'low/medium/high',
            'Multitasking':'low/medium/high',
            'Processing speed':'low/medium/high',
            'Budget':'numerical value'
            }}
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output 1: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input 2: {{'GPU intensity':     'low', 'Display quality':     'low', 'Portability':    'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90,000'}}
            output 2: {{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90000'}}

            input 3: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'
            output 3: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000'}}
            {delimiter}
            """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"""Here is the user input: {response}"""},
    ]

    confirmation = get_chat_completions(messages, json_format=True)

    return confirmation

## Stage 2

###  Implementing the Product Mapping


### `product_map_layer()`:

This function is responsible for extracting key features and criteria from laptop descriptions. Here's a breakdown of how it works:

-  Use a prompt that assign it the role of a Laptop Specifications Classifier, whose objective is to extract key features and classify them based on laptop descriptions.

- Provide step-by-step instructions for extracting laptop features from description.

- Assign specific rules for each feature (e.g., GPU Intensity, Display Quality, Portability, Multitasking, Processing Speed) and associate them with the appropriate classification value (Low, Medium, or High).

- Includes Few Shot Prompting (sample conversation between the user and assistant) to demonstrate the expected result of the feature extraction and classification process.

In [99]:
def product_map_layer(laptop_description):
    delimiter = "#####"

    lap_spec = {
        "GPU intensity": "(Type of the Graphics Processor)",
        "Display quality": "(Display Type, Screen Resolution, Display Size)",
        "Portability": "(Laptop Weight)",
        "Multitasking": "(RAM Size)",
        "Processing speed": "(CPU Type, Core, Clock Speed)",
    }

    values = {"low", "medium", "high"}

    prompt = f"""
    You are a Laptop Specifications Classifier whose job is to extract the key features of laptops and classify them as per their requirements.
    To analyze each laptop, perform the following steps:
    Step 1: Extract the laptop's primary features from the description {laptop_description}
    Step 2: Store the extracted features in {lap_spec} \
    Step 3: Classify each of the items in {lap_spec} into {values} based on the following rules: \
    {delimiter}
    GPU Intensity:
    - low: <<< if GPU is entry-level such as an integrated graphics processor or entry-level dedicated graphics like Intel UHD >>> , \n
    - medium: <<< if mid-range dedicated graphics like M1, AMD Radeon, Intel Iris >>> , \n
    - high: <<< high-end dedicated graphics like Nvidia RTX >>> , \n

    Display Quality:
    - low: <<< if resolution is below Full HD (e.g., 1366x768). >>> , \n
    - medium: <<< if Full HD resolution (1920x1080) or higher. >>> , \n
    - high: <<< if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. >>> \n

    Portability:
    - high: <<< if laptop weight is less than 1.51 kg >>> , \n
    - medium: <<< if laptop weight is between 1.51 kg and 2.51 kg >>> , \n
    - low: <<< if laptop weight is greater than 2.51 kg >>> \n

    Multitasking:
    - low: <<< If RAM size is 8 GB, 12 GB >>> , \n
    - medium: <<< if RAM size is 16 GB >>> , \n
    - high: <<< if RAM size is 32 GB, 64 GB >>> \n

    Processing Speed:
    - low: <<< if entry-level processors like Intel Core i3, AMD Ryzen 3 >>> , \n
    - medium: <<< if Mid-range processors like Intel Core i5, AMD Ryzen 5 >>> , \n
    - high: <<< if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher >>> \n
    {delimiter}

    {delimiter}
    Here is input output pair for few-shot learning:
    input 1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
    output 1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

    {delimiter}
    ### Strictly don't keep any other text in the values of the JSON dictionary other than low or medium or high ###
    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {lap_spec} for the following laptop {laptop_description}."""
    # see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input},
    ]

    response = get_chat_completions(messages, json_format=True)

    return response

In [91]:
laptop_df = pd.read_csv("laptop_data.csv")
laptop_df["laptop_feature"] = laptop_df["Description"].apply(
    lambda x: product_map_layer(x)
)

In [92]:
laptop_df.to_csv("updated_laptop.csv", index=False, header=True)

In [93]:
pd.read_csv("updated_laptop.csv").head(5)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,laptop_feature
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"{'GPU intensity': 'medium', 'Display quality':..."
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"{'GPU intensity': 'high', 'Display quality': '..."
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...,"{'GPU intensity': 'low', 'Display quality': 'm..."
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,"{'GPU intensity': 'low', 'Display quality': 'l..."
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,"{'GPU intensity': 'high', 'Display quality': '..."
5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.2 kg,Dual Cooling Fans,1 year,5 hours,"80,000",The Acer Predator is a powerhouse laptop desig...,"{'GPU intensity': 'high', 'Display quality': '..."
6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8GB,SSD,PixelSense,"13.5""",Intel Iris Plus,2256x1504,Windows 11,1.3 kg,Touchscreen Display,1 year,10 hours,"90,000",The Microsoft Surface Laptop is a premium devi...,"{'GPU intensity': 'medium', 'Display quality':..."
7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.6 kg,Backlit Keyboard,3 years,6 hours,"60,000",The Lenovo ThinkPad is a powerful laptop desig...,"{'GPU intensity': 'high', 'Display quality': '..."
8,HP,Pavilion,i5,Intel,2.3 GHz,12GB,HDD,LCD,"15.6""",Intel UHD,1366x768,Windows 10,2.1 kg,B&O Audio,1 year,4 hours,"30,000",The HP Pavilion is a budget-friendly laptop th...,"{'GPU intensity': 'low', 'Display quality': 'l..."
9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16GB,SSD,IPS,"17.3""",NVIDIA RTX,1920x1080,Windows 10,2.9 kg,Aura Sync RGB Keyboard,2 years,5 hours,"85,000",The ASUS ROG Strix G is a high-performance gam...,"{'GPU intensity': 'high', 'Display quality': '..."


In [94]:
debug_response_assistant_n = f"""Thank you for providing your budget.
Based on your budget of 50,000 INR, I will consider this while recommending suitable laptop options for you.
Here is the final recommendation for your laptop:
- GPU intensity: high
- Display quality: high
- Portability: low
- Multitasking: high
- Processing speed: medium
- Budget: 80,000 INR

Please note that these specifications are based on your requirements for surfing and a decent display within your budget.
Let me know if there's anything else I can assist you with!"""

In [95]:
debug_conversation = initialize_conversation()
debug_user_input = "Hi, I am Anvit. I need a laptop for coding."
debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_completions(debug_conversation)
debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
# print(debug_confirmation)
# After a series of conversation...
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '80000'}


### `compare_laptops_with_user()`:

This function compares the user's profile with the different laptops and come back with the top  recommendations. It will perform the following steps:
    - It will take the user requirements dictionary as input

    - Filter the laptops based on their price, keeping only the ones within the user's budget.

    - Calculate a score for each laptop based on how well it matches the user's requirements.

    - Sort the laptops based on their scores in descending order.

    - Return the top 3 laptops as a JSON-formatted string.

In [120]:
def compare_laptops_with_user(user_req_string):
    laptop_df = pd.read_csv("updated_laptop.csv")
    # user_requirements = response_dict_n
    user_requirements = user_req_string

    # Extracting user requirements from the input string (assuming it's a dictionary)
    # Since the function parameter already seems to be a string, we'll use it directly instead of extracting from a dictionary
    # Extracting the budget value from user_requirements and converting it to an integer
    budget = int(user_requirements.get("Budget", "0"))  # .replace(',', '').split()[0])
    # budget
    # # Creating a copy of the DataFrame and filtering laptops based on the budget
    filtered_laptops = laptop_df.copy()
    filtered_laptops["Price"] = (
        filtered_laptops["Price"].str.replace(",", "").astype(int)
    )
    filtered_laptops = filtered_laptops[filtered_laptops["Price"] <= budget].copy()
    # filtered_laptops
    # # # Mapping string values 'low', 'medium', 'high' to numerical scores 0, 1, 2
    mappings = {"low": 0, "medium": 1, "high": 2}

    # # # Creating a new column 'Score' in the filtered DataFrame and initializing it to 0
    filtered_laptops["Score"] = 0

    # # # Iterating over each laptop in the filtered DataFrame to calculate scores based on user requirements
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row["laptop_feature"]
        # laptop_values = user_product_match_str
        laptop_values = dictionary_present(user_product_match_str)
        score = 0

        #     # Comparing user requirements with laptop features and updating scores
        for key, user_value in user_requirements.items():
            # if key.lower() == 'budget':
            if key == "Budget":
                continue  # Skipping budget comparison
            laptop_value = laptop_values.get(key, None)
            # print(key, laptop_value)
            laptop_mapping = mappings.get(laptop_value, -1)
            # laptop_mapping = mappings.get(laptop_value, -1)
            # user_mapping = mappings.get(user_value, -1)
            user_mapping = mappings.get(user_value, -1)
            if laptop_mapping >= user_mapping:
                score += (
                    1  # Incrementing score if laptop value meets or exceeds user value
                )

        filtered_laptops.loc[index, "Score"] = (
            score  # Updating the 'Score' column in the DataFrame
        )

    # Sorting laptops by score in descending order and selecting the top 3 products
    top_laptops = filtered_laptops.drop("laptop_feature", axis=1)
    top_laptops = top_laptops.sort_values("Score", ascending=False).head(3)
    top_laptops_json = top_laptops.to_json(
        orient="records"
    )  # Converting the top laptops DataFrame to JSON format

    # top_laptops
    return top_laptops_json

## Stage 3

### 3.4: Product Recommendation Layer

Finally, we come to the product recommendation layer. It takes the output from the `compare_laptops_with_user` function in the previous layer and provides the recommendations to the user. It has the following steps.
1. It continues the conversation for recommendation.
2. Generate the recommendations and display in a presentable format.
3. Ask questions basis the recommendations.



In [2]:
functions_list = [
    {
        "name": "compare_laptops_with_user",
        "description": "Compare laptops with user requirement",
        "parameters": {
            "type": "object",
            "properties": {
                "user_req_string": {
                    "type": "string",
                    "description": "Takes the input from the user",
                }
            },
            "required": ["user_req_string"],
        },
    }
]
system_message = (
    f"""Your sole purpose is to call the function compare_laptops_with_user"""
)


def get_chat_completions_using_function_calling(user_req_string):
    messages_payload = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": str(user_req_string)},
    ]
    chat_completions = client.chat.completions.create(
        model="gpt-4o",
        messages=messages_payload,
        seed=2345,
        functions=functions_list,
        function_call="auto",
    )
    output = chat_completions.choices[0].message
    if output.function_call:
        function_name = output.function_call.name
        function_args = output.function_call.arguments
        print(f"Function Name: {function_name}")
        # print(function_args)
        if function_name == "compare_laptops_with_user":
            output = compare_laptops_with_user(
                ast.literal_eval(ast.literal_eval(function_args)["user_req_string"])
            )

    return output

In [122]:
def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]["Score"] > 2:
            data1.append(data[i])

    return data1

In [ ]:
top_3_laptops = compare_laptops_with_user(response_dict_n)

display(top_3_laptops)

In [ ]:
# Get output in JSON Format
top_3_laptops_json = json.loads(top_3_laptops)
# type(top_3_laptops_json)
top_3_laptops_json

In [123]:
# def initialize_conv_reco(products):
#     system_message = f"""
#     You are an intelligent laptop gadget expert and you are tasked with the objective to \
#     solve the user queries about any product from the catalogue in the user message \
#     You should keep the user profile in mind while answering the questions.\
#
#     Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
#     1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
#     2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
#
#     """
#     user_message = f""" These are the user's products: {products}"""
#     conversation = [
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": user_message},
#     ]
#     # conversation_final = conversation[0]['content']
#     return conversation

In [ ]:
# debug_conversation_reco = initialize_conv_reco(top_3_laptops)
# debug_conversation_reco

In [ ]:
# debug_recommendation = get_chat_completions(debug_conversation_reco)
# print(debug_recommendation + "\n")

## Combining all the 3 stages

Now, we combine all the three stages that we defined above.

`Stage 1` + `Stage 2` + `Stage 3`

### 3.5 Dialogue Management System

Bringing everything together, we create a `diagloue_mgmt_system()` function that contains the logic of how the different layers would interact with each other. This will be the function that we'll call to initiate the chatbot

In [125]:
def dialogue_mgmt_system():
    # initialize the conversation
    conversation = initialize_conversation()
    # First message from Assistant
    print("Assistant:\nHow may I help you with your laptop selection?\n")
    # introduction = get_chat_completions(conversation)

    user_input = ""

    # Loop till user exits
    while user_input.lower() != "exit":
        print("User: ", end=" ")
        user_input = input()
        print(user_input, end="\n")
        # Moderation Check for user input
        moderation = moderation_check(user_input)
        if moderation == "Flagged":
            print(
                "\nAssistant:\nSorry, this message has been flagged. Please restart your conversation.\n"
            )
            break

        if user_input.lower() == "exit":
            print(
                "\nAssistant:\nThank you for using this service. Please do not hesitate to contact us if you need assistance. See you soon!\n"
            )
            break

        conversation.append({"role": "user", "content": user_input})
        response = get_chat_completions_using_function_calling(conversation)

        # Moderation Check for LLM response
        moderation = moderation_check(response[0]["content"])
        if moderation == "Flagged":
            print(
                "\nAssistant:\nSorry, this message has been flagged. Please restart your conversation.\n"
            )
            break

        print("\nAssistant:\n", response[0]["content"], "\n")

        conversation += response

In [126]:
dialogue_mgmt_system()

Assistant:
How may I help you with your laptop selection?

User:  I am a gamer


Assistant:
 Great! As a gamer, you would likely require a laptop with high GPU intensity for smooth gameplay. You would also need a high-quality display for the best gaming experience. 

Could you please tell me more about the types of games you play? Are they high-end games with heavy graphics or more casual games? This will help me understand your processing speed and multitasking requirements. 

Also, do you travel often with your laptop for gaming tournaments or do you primarily game from home? This will help me understand your portability needs. 

Lastly, could you please share your budget for the new laptop? 

User:  i prefer working at home

Assistant:
 Thank you for sharing that. It seems like portability might not be a high priority for you. 

Could you please tell me more about the types of games you play? Are they high-end games with heavy graphics or more casual games? This will help me underst